### Practice 3
Dealing with missing values
- Imputing
- Dropping data

In [1]:
import pandas as pd
import numpy as np

In [2]:
car_sales_missing = pd.read_csv('data/car-sales-extended-missing-data.csv')
car_sales_missing.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0


In [3]:
car_sales_missing.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

### Impute data via Pandas

In [4]:
# Create copy
car_sales_filled_1 = car_sales_missing.copy()

In [5]:
# Impute data
car_sales_filled_1['Make'].fillna('missing', inplace=True)
car_sales_filled_1['Colour'].fillna('missing', inplace=True)
car_sales_filled_1['Odometer (KM)'].fillna(car_sales_filled_1['Odometer (KM)'].mean(), inplace=True)
car_sales_filled_1['Doors'].fillna(4, inplace=True)

# Drop missing targets
# Imputing should not be considered for target columns
car_sales_filled_1.dropna(inplace=True)

In [6]:
car_sales_filled_1

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0
...,...,...,...,...,...
995,Toyota,Black,35820.0,4.0,32042.0
996,missing,White,155144.0,3.0,5716.0
997,Nissan,Blue,66604.0,4.0,31570.0
998,Honda,White,215883.0,4.0,4001.0


In [7]:
car_sales_filled_1.isna().sum()

Make             0
Colour           0
Odometer (KM)    0
Doors            0
Price            0
dtype: int64

In [8]:
# Encode non-numerical attributes
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

cat_features = ['Make', 'Colour', 'Doors']
one_hot = OneHotEncoder()

transformer = ColumnTransformer([
    ('one_hot', one_hot, cat_features)
], remainder='passthrough')

# Transformed DF includes target column 'Price'
transformed_car_sales_1 = pd.DataFrame(transformer.fit_transform(car_sales_filled_1))
transformed_car_sales_1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,35431.0,15323.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,192714.0,19943.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,84714.0,28343.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,154365.0,13434.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,181577.0,14043.0


In [9]:
transformed_car_sales_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       950 non-null    float64
 1   1       950 non-null    float64
 2   2       950 non-null    float64
 3   3       950 non-null    float64
 4   4       950 non-null    float64
 5   5       950 non-null    float64
 6   6       950 non-null    float64
 7   7       950 non-null    float64
 8   8       950 non-null    float64
 9   9       950 non-null    float64
 10  10      950 non-null    float64
 11  11      950 non-null    float64
 12  12      950 non-null    float64
 13  13      950 non-null    float64
 14  14      950 non-null    float64
 15  15      950 non-null    float64
dtypes: float64(16)
memory usage: 118.9 KB


### Impute with scikit-learn

In [10]:
# Work on another copy of dataframe!
car_sales_filled_2 = car_sales_missing
car_sales_filled_2.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431.0,4.0,15323.0
1,BMW,Blue,192714.0,5.0,19943.0
2,Honda,White,84714.0,4.0,28343.0
3,Toyota,White,154365.0,4.0,13434.0
4,Nissan,Blue,181577.0,3.0,14043.0


In [11]:
car_sales_filled_2.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [12]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [13]:
cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')
door_imputer = SimpleImputer(strategy='constant', fill_value=4)
num_imputer = SimpleImputer(strategy='mean')

# Define columns
cat_features = ['Make', 'Colour']
door_feature = ['Doors']
num_features = ['Odometer (KM)']

# Create imputer
imputer = ColumnTransformer([
    ('cat_imputer', cat_imputer, cat_features),
    ('door_imputer', door_imputer, door_feature),
    ('num_imputer', num_imputer, num_features)
])

# Transform data
car_sales_filled_2 = pd.DataFrame(imputer.fit_transform(car_sales_filled_2), columns=['Make', 'Colour', 'Doors', 'Odometer (KM)'])
car_sales_filled_2.head()

,Make,Colour,Doors,Odometer (KM)
0,Honda,White,4,35431
1,BMW,Blue,5,192714
2,Honda,White,4,84714
3,Toyota,White,4,154365
4,Nissan,Blue,3,181577


In [14]:
# NOTE: car_sales_filled_2 DF does not have target column
car_sales_filled_2

,Make,Colour,Doors,Odometer (KM)
0,Honda,White,4,35431
1,BMW,Blue,5,192714
2,Honda,White,4,84714
3,Toyota,White,4,154365
4,Nissan,Blue,3,181577
...,...,...,...,...
995,Toyota,Black,4,35820
996,missing,White,3,155144
997,Nissan,Blue,4,66604
998,Honda,White,4,215883


In [15]:
car_sales_filled_2.isna().sum()

Make             0
Colour           0
Doors            0
Odometer (KM)    0
dtype: int64

In [16]:
# Encode non-numerical attributes
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

cat_features = ['Make', 'Colour', 'Doors']
one_hot = OneHotEncoder()

transformer2 = ColumnTransformer([
    ('one_hot', one_hot, cat_features)
], remainder='passthrough')

transformed_car_sales_2 = pd.DataFrame(transformer2.fit_transform(car_sales_filled_2.reset_index()))
transformed_car_sales_2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,35431
1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,192714
2,0,1,0,0,0,0,0,0,0,1,0,0,1,0,2,84714
3,0,0,0,1,0,0,0,0,0,1,0,0,1,0,3,154365
4,0,0,1,0,0,0,1,0,0,0,0,1,0,0,4,181577


In [17]:
transformed_car_sales_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1000 non-null   object
 1   1       1000 non-null   object
 2   2       1000 non-null   object
 3   3       1000 non-null   object
 4   4       1000 non-null   object
 5   5       1000 non-null   object
 6   6       1000 non-null   object
 7   7       1000 non-null   object
 8   8       1000 non-null   object
 9   9       1000 non-null   object
 10  10      1000 non-null   object
 11  11      1000 non-null   object
 12  12      1000 non-null   object
 13  13      1000 non-null   object
 14  14      1000 non-null   object
 15  15      1000 non-null   object
dtypes: object(16)
memory usage: 125.1+ KB
